In [1]:
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import os
import openpyxl
import re
import wordcloud
import jieba
import jieba.analyse
import jieba.posseg
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = ['sans-serif'] 
plt.rcParams['font.sans-serif'] = ['SimHei']
import mlxtend
from mlxtend.frequent_patterns import apriori  #挖掘频繁项集
from mlxtend.frequent_patterns import association_rules #挖掘关联规则
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
data = pd.read_excel(r'E:\Ajiao\notebook\notebook\事故调查报告列表-中远海重.xls')
data

,序号,企业名称,部门,事件名称,报告时间,事件发生日期,发生时间,项目名称,地点选取,IMO number,...,肇事人本工种工龄,肇事人入职时间,肇事人接受教育时间,肇事人性别,肇事人婚姻状况,肇事人是否为新工人,肇事人如新工人有无船厂工作经验,肇事人工种,肇事人用工形式,肇事人文化程度
0,1,上海中远海运重工,服务工区,机械伤害,2020-04-21 10:35,2020-04-20,17:30,“海昌”轮,船舶,NaN,...,1,2019-11-08,2019-11-04,男,已婚,新工人,无,起重作业指挥,外包制,初中
1,2,扬州中远海运重工,NaN,内业科物体打击,2020-04-10 08:01,2020-04-06,18:00,T排焊接工场,内场,NaN,...,3,2018-11-08,2018-11-08,男,已婚,老工人,NaN,辅助工,协力制,初中
2,3,大连中远海运川崎,NaN,吊物牵引绳夹具飞出击伤作业者脸部！,2020-04-02 10:00,2020-03-31,16:10,NaN,内场,NaN,...,10,2010-07-01,2010-07-01,男,已婚,老工人,NaN,装配工,合同制,高中、职专、中技
3,4,扬州中远海运重工,NaN,千斤顶坠落险肇,2020-04-02 17:01,2020-03-30,09:20,N937,码头或坞,NaN,...,3,2020-03-28,2020-03-28,男,已婚,新工人,有,装配工,外包制,初中
4,5,扬州中远海运重工,NaN,内业科 104#电磁桥式起重机钢板掉落险肇,2020-04-05 08:32,2020-03-26,10:30,NaN,内场,NaN,...,3,2019-09-24,2019-09-24,女,已婚,老工人,NaN,起重司机,协力制,高中、职专、中技
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,1980,南通船务,服务车间（修）,涂装车间工人物体打击死亡,2002-03-11 14:45,2002-03-11,01:00,德鲁莎,码头或坞,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,1981,南通船务,船体车间（修）,2002.2.23船体车间马某高空坠落,2002-02-23 14:35,2002-02-23,14:36,兰博,码头或坞,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981,1982,南通船务,船体车间（修）,船体车间三工段尤某装配作业死亡,2001-09-14 14:28,2001-09-14,17:00,塔波拉,码头或坞,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,1983,南通船务,船体车间（修）,2001.7.10船体车间王某叉车致死,2001-07-10 14:20,2001-07-10,14:30,内场,内场,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
branc =pd.read_csv(r'E:\基于文本挖掘的船舶建造事故致因分析\brach.csv',encoding='utf_8_SIG')
branc

In [ ]:
b = branc.values.tolist()
b

In [ ]:
#第二种：原始数据集已经拆分，输出支持度
support_data,all_freq_transaction=apriori(b,min_support=0.005)
support_1=pd.DataFrame([support_data])#支持度
support_1.stack().to_frame()

In [3]:
#辅助函数1
def create_c1(dataset):
    '''
    函数：生成第一个候选集cl,每个项集只有一个item
    参数：dataset是原始数据集
    返回：frozenset形式的候选集合cl
    '''
    c1=[]
    #遍历每一条交易
    for transaction in dataset:
        for item in transaction:    
            #去重
            if not {item} in c1:
                c1.append({item})
    c1.sort()#对原列表进行排序
    return list(map(frozenset,c1)) #转换成集合   #不能转换成集合，集合是无序的。

In [4]:
#辅助函数2
def create_freq_transaction(dataset,ck,min_support=0.1):
        '''
        函数：生成满足最小支持度的频繁项集
        参数：dataset 原始数据集
              ck:候选项集
              min_support:最小支持度
        返回： support_data:候选项集ck支持度字典（key候选项，value支持度）
               freq_transaction:给定min_support 下的ck中频繁项集
        注意：如果ck中得不到频繁项集，则返回的是空list
        '''
        sscnt={}#存放候选项集及频次
        for transaction in dataset:#遍历交易的每一条数据
            for can in ck:#候选集的每一条数据
                if can.issubset(transaction):#判断是不是子集，如果是就+1
                    sscnt[can] = sscnt.get(can,0)+1
        #事物总量：原始数据集的长度            
        num_transactions = float(len(dataset))
        #频繁项集，如果ck中得不到频繁项集，则返回的是空list
        freq_transaction = []
        #存放频繁项集及支持度的集合
        support_data ={}
        for key in sscnt:#遍历候选项集，计算支持度
            support = sscnt[key]/num_transactions
            support_data[key]=support
            if support>=min_support:#满足最小支持度，则保存到频繁项集中去
                freq_transaction.append(key)
            else:
                support_data.pop(key)
        return support_data,freq_transaction#返回支持度和频繁项集

In [5]:
#辅助函数3
def create_ck(freq_k_transaction):
    '''
    函数：由频繁K项集生成k+1候选项集
    参数：
    freq_k_transaction:频繁k项集(函数输入是一个频繁项集)
    这里不是返回频繁K项集，而非k项集，体现了先验原理。即如果一个项集是频繁的，则他的所有自己也一定是频繁的。
    反之，如果一个项集是非频繁项集，那么他的所有超集也是非频繁的。
    返回：
    ck:k+1候选项集
   这里不是返回频繁k+1项集，暂时没有用支持度阈值对结果过滤。

    #特别重要，体现算法原理，节约计算资源，有3个关键点。
    1、不是从k候选集生成k+1候选项集。目标是频繁项集，非频繁项集能不计算就不计算，节约资源。
    2、频繁k项集两两结合。为何不是K-频繁项集和1-候选项集两两结合，是为了节约计算资源。先验原理告诉我们，这是可行的。
    3、只保留k+1候选项集，逐步计算。
    '''
    #k+1候选项集
    ck=[] 
    k = len(freq_k_transaction)
    for i in range(k):
        for j in range(i+1,k): #遍历频繁项集，两两组合
            t1 = freq_k_transaction[i]
            t2 = freq_k_transaction[j]
            t =  t1|t2    
            #两两组合后的长度是k+1，且两两组合后的项集不在k+1候选项集中。
            if (not t in ck) and (len(t)==len(freq_k_transaction[0])+1):
                ck.append(t)
    return ck

In [6]:
#主函数
def apriori(dataset,min_support=0.1):
    '''
    函数：生成大于支持度阈值下的频繁项集，以及所有候选项集及其支持度
    参数：
    dataset：原始数据事物集
    min_support:最小支持度
    返回：
    support_data:所有候选项集ck的支持度字典（key候选项集，value支持度）
        all_freq_transaction:给定min_support下的所有频繁项集
    '''
    #一项候选项集
    c1=create_c1(dataset)
    #一项频繁项集
    support_data,freq_transaction_1=create_freq_transaction(dataset,c1,min_support=min_support)
    #保存到总频繁项集
    all_freq_transaction=[freq_transaction_1]
    #初始化，辅助参数，没有实际意义
    k=2
    #最后一项非空时
    while len(all_freq_transaction[-1])>0:
        ck = create_ck(all_freq_transaction[-1])#最后一个频繁项集
        support_data_k,freq_transaction_k=create_freq_transaction(dataset,ck,min_support=min_support)
        #更新字典，批量增加，更新支持度
        support_data.update(support_data_k)
        #生成新的频繁项集加到总的频繁项集
        all_freq_transaction.append(freq_transaction_k)
        k=k+1
    return support_data,all_freq_transaction  #返回支持度和所有的频繁项集
   

In [7]:
#辅助函数1：给定一个集合，计算出所有子集
def create_subset(fromlist,tolist):
    '''
    函数：已知列表fromlist的所有子集，结果保存到tolist,tolist中的元素格式是冻集合。
    先生成k-1的一个子集，在生成k-2的一个子集
    '''
    for i in range(len(fromlist)):
        t = [fromlist[i]]#先得到k-1项的一个子集
        tt = frozenset(set(fromlist)-set(t))
        if not tt in tolist:
            tolist.append(tt)
            
            tt=list(tt)
            if len(tt)>1:
                create_subset(tt,tolist)
    return None

In [8]:
#辅助函数2
def cal_conf(fre_set,h,support_data,rulelist,min_conf):
    '''
    函数：给定一个频繁项集，得到它子集中所有可能的关联规则，满足置信度阈值的强关联规则保存在rulelist中。
    参数：h是fre_set（频繁项集）的所有子集集合
    提升度需要大于1，rulelist是关联规则
    
    置信度：x y的联合概率除以x的概率
    将辅助函数之前，先看置信度公式
    '''
    prunedH=[]#记录置信度大于阈值的集合
    for after in h:#从子集中找后键
        #计算以after为后项以fre_set-after为前项的置信度
        conf = support_data[fre_set]/support_data[fre_set-after]#置信度
        lift = support_data[fre_set]/(support_data[fre_set-after]*support_data[after])#提升度
        if conf>=min_conf and lift>1:#只保留提升度大于1，置信度大于阈值的部分
            print(fre_set-after,'-->',after,'\n',#打印所有前键和后键
                  'before support:',round(support_data[fre_set-after],3),',',#前键的支持度
                 'after support:',round(support_data[after],3),',',#后键的支持度
                 'together support:',round(support_data[fre_set],3),',',#联合支持度也就是联合概率
                 'conf:',round(conf,3),',',
                 'lift:',round(lift,3))
            #以元组形式保存强关联规则到rulelist中
            rulelist.append((fre_set-after,after,round(support_data[fre_set],3),round(conf,3),round(lift,3)))
            prunedH.append(after)
    return prunedH

In [9]:
#主函数
def create_rules(support_data,all_freq_transaction,min_conf=0.08):
    '''
    函数：生成给定频繁项集下的所有关联规则项。
    '''
    all_rulelist=[]#总的关联规则的参数
    for i in range(1,len(all_freq_transaction)):
        for fre_set in all_freq_transaction[i]:#遍历所有频繁项集
            #求k项集的所有非空子集，1项集，2项集，直到k-1项集，用h表示，为list类型，元素为frozenset类型
            fre_list=list(fre_set)
            all_subset=[]#得到所有子集
            create_subset(fre_list,all_subset)
            cal_conf(fre_set,all_subset,support_data,all_rulelist,min_conf)
    return all_rulelist